# Knihovny

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Načtení CSV souborů

In [440]:
clicks = pd.read_csv('https://raw.githubusercontent.com/PetrNovak96/ck_poznani/master/data/clicks.csv',sep=',')
visits = pd.read_csv('https://raw.githubusercontent.com/PetrNovak96/ck_poznani/master/data/visitors.csv',sep=';')
streams = pd.read_csv('https://raw.githubusercontent.com/PetrNovak96/ck_poznani/master/data/Clickstreams.csv',sep=';')
se_map = pd.read_csv('https://raw.githubusercontent.com/PetrNovak96/ck_poznani/master/data/search_engine_map.csv',sep=',')

# Průměrný Conversion rate a bounce rate

In [441]:
# conversion rate
conv_visitIDs = clicks.loc[clicks['PageName'] == 'n_katalog.asp'].VisitID.unique()
all_visitIDs = clicks.VisitID.unique()
conv_rate = conv_visitIDs.shape[0] / all_visitIDs.shape[0]
print("Průměrný converion rate:\t{}".format(round(conv_rate,4)))

# bounce rate
bounce_visitIDs = visits.loc[visits['Delka_pocetstranek'] == 1].VisitID
bounce_rate = bounce_visitIDs.shape[0] / all_visitIDs.shape[0]
print("Průměrný bounce rate:\t\t{}".format(round(bounce_rate,4)))

Průměrný converion rate:	0.0398
Průměrný bounce rate:		0.5729


# "Bounce pages"

In [442]:
bounce_clicks = clicks.loc[clicks['VisitID'].isin(bounce_visitIDs)]
bounce_pages = bounce_clicks.groupby(['PageName']).size().sort_values(ascending=False)
bounce_pages = bounce_pages.to_frame()
bounce_pages.columns = ['count']
bounce_pages
#bounce_pages.head(10)

,count
PageName,
www.poznani.cz,2597
poznani.cz,198
lastminute,116
bulharsko,106
korsika,96
...,...
srbsko-eerná-hora-bosna-a-hercegovina-poklady-a-tajemství-jugoslávského-balkánu-dovolená-2005.htm,1
srbsko-černá-hora-bosna-a-hercegovina-poklady-a-tajemství-jugoslávského-balkánu.htm,1
irsko-wales-irsko-země-keltských-tradic-a-přírodní-parky-walesu-dovolená-a-turistika-2005.htm,1


# Vztah typu refereru vůči bounce a convergence rate

In [443]:
visits['Bounce'] = visits['VisitID'].isin(bounce_visitIDs).astype(int)
visits['Konverze'] = visits['VisitID'].isin(conv_visitIDs).astype(int)

merged = pd.merge(
    visits[['VisitID','Referrer','Bounce','Konverze']],
    se_map[['Referrer','TypOdkazovace']],
    on='Referrer')[['VisitID','Referrer','TypOdkazovace','Bounce','Konverze']] 

grouped = merged.drop(['VisitID','Referrer'],axis=1).groupby(['TypOdkazovace']).agg({'TypOdkazovace': ['count'], 'Bounce': ['sum'],'Konverze': ['sum']})
grouped['Bounce rate %'] = round((grouped['Bounce']['sum'] / grouped['TypOdkazovace']['count'])*100,2)
grouped['Convergence rate %'] = round( (grouped['Konverze']['sum'] / grouped['TypOdkazovace']['count'])*100,2)
grouped

,TypOdkazovace,Bounce,Konverze,Bounce rate %,Convergence rate %
,count,sum,sum,,
TypOdkazovace,,,,,
Fulltext,7168,4921,168,68.65,2.34
Neni,5821,2790,314,47.93,5.39
OstatniNerozliseno,402,160,17,39.80,4.23
Partneri,580,278,39,47.93,6.72
Vlastni Weby,824,558,12,67.72,1.46
katalog,764,208,70,27.23,9.16


# Tabulka korelací

In [444]:
merged.drop(['VisitID'],axis=1).assign(TypOdkazovace=merged.TypOdkazovace.astype('category').cat.codes).corr()

,TypOdkazovace,Bounce,Konverze
TypOdkazovace,1.000000,-0.156028,0.058699
Bounce,-0.156028,1.000000,-0.188155
Konverze,0.058699,-0.188155,1.000000
